In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install console-progressbar

In [ ]:
!pip install fastai==1.0.61

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
from fastai.metrics import error_rate
from fastai.vision.data import ImageDataBunch
from fastai import *
import cv2 as cv
import numpy as np
import pandas as pd
import scipy.io as sio
import zipfile



#Reference taken for the code 
# dhamvi01(2019),FastAI-Image-Classification
# https://github.com/dhamvi01/FastAI-Image-Classification/blob/master/fastai.ipynb

# Data prprocessing

# Prepare data batches

In [ ]:

data = ImageDataBunch.from_folder('/content/drive/MyDrive/cars_classification/Data/car_data/car_data','train','valid',ds_tfms=get_transforms(do_flip=False, flip_vert=True, max_rotate=5.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75),size=224,bs=32).normalize(imagenet_stats)

In [ ]:
# show the batches of image tha created by the model itself...
data.show_batch()

In [ ]:
# class names and number of classes
print(data.classes)
len(data.classes),data.c

# # Training: Alexnet
# For the training, using Alexnet and batch size of 32.

In [ ]:
epochs = 40
learn = cnn_learner(data, models.alexnet, metrics=accuracy)
learn.fit_one_cycle(epochs)


data = learn.recorder.plot_losses()
data_acc = learn.recorder.plot_metrics()

steps_per_epoch = int(len(data[1])/epochs)
# print(steps_per_epoch)
training_loss = []
for st in range(epochs+1):
    if st == 0:
        training_loss.append(data[1][st])
    else:
        training_loss.append(data[1][(st*steps_per_epoch)-1])

total_epochs = [i for i in range(epochs+1)]
epochs_val = total_epochs.copy()
del epochs_val[0]
val_loss = data[-1]

# Ploting loss and accuracy graphs

In [ ]:

figure, axis = plt.subplots(2, 2)
# For Train Loss
axis[0, 0].plot(total_epochs, training_loss, label='Train Loss')
axis[0, 0].set_title("Train Loss")
# For Test Loss
axis[0, 1].plot(epochs_val, val_loss, label='Test Loss')
axis[0, 1].set_title("Test Loss")
# For Test Loss
axis[1, 0].plot(epochs_val, data_acc, label='Test Accuracy')
axis[1, 0].set_title("Test Accuracy")
# For Overall Model Evaluation
axis[1, 1].plot(total_epochs, training_loss, label='Train Loss')
axis[1, 1].plot(epochs_val, val_loss, label='Test Loss')
axis[1, 1].plot(epochs_val, data_acc, label='Test Accuracy')
axis[1, 1].set_title("All Training Plots")
axis[1, 1].legend()
plt.savefig('AlexNet_fastai_plot.png')

In [ ]:
# calculate the accuracy
preds,y, loss = learn.get_preds(with_loss=True)
acc = accuracy(preds, y)
print('The accuracy is {0} %.'.format(acc))


# Save and Export The model

In [ ]:

learn.save('/content/drive/MyDrive/cars_classification/models_alex_net/stanford-cars-1')
learn.export('/content/drive/MyDrive/cars_classification/models_alex_net/export.pkl')